In [1]:
from RunGerryChainDefs import *

In [2]:
(graph, chain) = GerryChainSetUp(Input_Shp='/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/Alabama shp files/Alabama_VTD_District_Intersection.shp',
                                        Total_Steps=10)
(chain_list, meta_list, part_dict, seg_output) = RunGerryChain(graph, chain, Checkpoint=2, VTD_Data = '/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/Agregate VTD Demographic Data/AL.csv', Output_Csv='/Users/rebeccawagner/Desktop/csv', Output_Metadata='/Users/rebeccawagner/Desktop/meta', Output_Seg='/Users/rebeccawagner/Desktop/seg')

/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:21: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometries[i].id = i
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/adjacency.py:23: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  tree = STRtree(geometries)
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/graph.py:162: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
/Users/rebeccawagner/opt/anaconda3/envs/gerry/lib/python3.10/site-packages/gerrychain/graph/graph.py:272: UserWarning: NA values found in column last!
  warnings.wa

0
Outputing Data
0
Outputing Data
0
Outputing Data
0
Outputing Data
0
Outputing Data
0
Outputing Data
0

Chain complete. Time elapsed: 7.481223821640015 seconds.
0


In [6]:
pd.read_csv("/Users/rebeccawagner/Desktop/csv_10all")

,Iteration,1,2,3,4,5,6,7,8,9,10
0,01003000001_01001,1001,1007,1007,1007,1007,1007,1007,1007,1007,1007
1,01003000002_01001,1001,1007,1007,1007,1007,1007,1007,1007,1007,1007
2,01003000003_01001,1001,1007,1007,1007,1007,1007,1007,1007,1007,1007
3,01003000004_01001,1001,1007,1007,1007,1007,1007,1007,1007,1007,1007
4,01003000005_01001,1001,1007,1007,1007,1007,1007,1007,1007,1007,1007
...,...,...,...,...,...,...,...,...,...,...,...
1839,01131005006_01007,1007,1001,1001,1001,1001,1001,1006,1006,1006,1006
1840,01131006001_01007,1007,1001,1001,1001,1001,1001,1006,1006,1006,1006
1841,01131006002_01007,1007,1001,1001,1001,1001,1001,1006,1006,1006,1006
1842,01131006004_01007,1007,1001,1001,1001,1001,1001,1006,1006,1006,1006


In [36]:
districts = list(df[2].unique())
district_id = df[2] == districts[0]
vtds = list(df[district_id].index)
vtds

['01003000001_01001',
 '01003000002_01001',
 '01003000003_01001',
 '01003000004_01001',
 '01003000005_01001',
 '01003000006_01001',
 '01003000007_01001',
 '01003000008_01001',
 '01003000009_01001',
 '01003000010_01001',
 '01003000011_01001',
 '01003000012_01001',
 '01003000013_01001',
 '01003000014_01001',
 '01003000015_01001',
 '01003000016_01001',
 '01003000017_01001',
 '01003000018_01001',
 '01003000019_01001',
 '01003000020_01001',
 '01003000021_01001',
 '01003000022_01001',
 '01003000023_01001',
 '01003000024_01001',
 '01003000025_01001',
 '01003000026_01001',
 '01003000027_01001',
 '01003000028_01001',
 '01003000029_01001',
 '01003000030_01001',
 '01003000031_01001',
 '01003000032_01001',
 '01003000033_01001',
 '01003000034_01001',
 '01003000035_01001',
 '01003000036_01001',
 '01003000037_01001',
 '01003000038_01001',
 '01003000039_01001',
 '01003000040_01001',
 '01003000041_01001',
 '01003000042_01001',
 '01003000043_01001',
 '01003000044_01001',
 '01003000045_01001',
 '01003000

In [45]:
segregation_output = pd.DataFrame(index=districts)
score_list = []

for district in districts:
    dist_id = df["2"] == district
    vtd_list = list(df[district_id].index)
    seg_score = CalculateSegregation(data_file_path='/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/Agregate VTD Demographic Data/AL.csv',
                            vtds = vtd_list)
    score_list.append(seg_score)

segregation_output[1] = score_list


In [46]:
segregation_output

,1
1001,-0.449858
1002,-7.150571
1003,-3.792898
1004,-10.025746
1005,0.198728
1006,-14.170530
1007,18.428387


In [2]:
from GetLocalVTDEnv import *

In [3]:
GetVTDLocalEnv(Block_to_Vtd = '/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/Block to VTD/block_to_vtd_AL.csv',
                Block_LE_Data = '/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/Block Local Environments by State/AL_LE',
                save=True, save_path='/Users/rebeccawagner/Desktop/AL')

/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/GetLocalVTDEnv.py:28: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_df = merged_df.groupby(by="block_vtda",as_index=False).sum()[["block_vtda","total","nh_black","weighted_blk_sl_dist"]]


,block_vtda,total,nh_black,weighted_blk_sl_dist
0,01001000001,795,284,28065.174909
1,01001000002,4857,652,189112.957346
2,01001000003,1008,67,34712.651255
3,01001000004,10255,2270,397691.173409
4,01001000005,1192,242,41125.762503
...,...,...,...,...
1832,01133000007,2450,5,25549.047995
1833,01133000008,366,2,3514.336233
1834,01133000009,1102,4,21745.363824
1835,01133000010,824,2,8857.189811


In [25]:
Block_to_Vtd = '/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/Block to VTD/block_to_vtd_AL.csv'
Block_LE_Data = '/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/Data/Block Local Environments by State/AL_LE'

In [51]:
block_to_vtd = pd.read_csv(Block_to_Vtd, dtype={'GEOID20': str})[["GEOID20","block_vtda"]] # keep only necessary columns

# local environment data for block
block_le_data = pd.read_csv(Block_LE_Data, dtype={'block_geoid_start':str})

In [52]:
# merge dataframes
merged_df = block_to_vtd.merge(right=block_le_data, left_on='GEOID20', right_on='block_geoid_start')
#merged_df[["GEOID20","block_vtda","rn_total_sl_dist","rn_nh_black_sl_dist","total","nh_black"]]


In [35]:
block_le_data["block_geoid_start"][0]

'010010201001000'

In [37]:
block_to_vtd["GEOID20"][0]

'10010201001000'

In [53]:
merged_df

,GEOID20,block_vtda,block_geoid_end,block_geoid_start,sl_dist,rn_total_sl_dist,rn_hisp_sl_dist,rn_nh_white_sl_dist,rn_nh_black_sl_dist,state_fips,...,hisp,nh_white,nh_black,nh_na,nh_asn,nh_haw,nh_oth,nh_multi,block_id_crosswalk,_merge
0,010010201001000,01001000016,11170306162003,010010201001000,56.417145,717882,31053,370281,279291,1,...,0,12,4,0,0,0,0,5,1,matched (3)
1,010010201001001,01001000016,11170306071000,010010201001001,56.529419,717846,31079,370178,279322,1,...,1,18,11,0,2,0,0,2,2,matched (3)
2,010010201001002,01001000016,10810406052002,010010201001002,56.540180,717895,31190,369734,280087,1,...,0,24,2,0,0,0,2,1,3,matched (3)
3,010010201001003,01001000016,11170306072001,010010201001003,56.544136,717796,31287,369748,279913,1,...,0,16,0,0,0,0,0,1,4,matched (3)
4,010010201001006,01001000016,11210111003073,010010201001006,56.442619,717740,31359,370096,279534,1,...,1,1,5,0,0,0,0,0,5,matched (3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128361,011339659003065,01133000003,10330207041033,011339659003065,51.071598,717734,41695,484774,156877,1,...,0,3,0,0,0,0,0,0,128362,matched (3)
128362,011339659003068,01133000003,10730052003018,011339659003068,51.396698,717788,41697,486304,155137,1,...,3,18,0,0,0,0,0,0,128363,matched (3)
128363,011339659003069,01133000003,10730027022017,011339659003069,51.562206,717756,41263,488767,153031,1,...,1,2,0,0,0,0,0,0,128364,matched (3)
128364,011339659003070,01133000003,10730138013039,011339659003070,53.221428,717712,38930,511803,131191,1,...,0,9,0,0,0,0,0,1,128365,matched (3)
